In [1]:
import pandas as pd, sqlite3 as s3

In [2]:
conn = s3.connect("archive (2)/database.sqlite")

This section will focus on the Player Table from archive 2. This table lists information on baseball players' name, DOB, country, and playstyle

In [3]:
c = conn.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()
for l in c: print(l)

('Player',)
('Extra_Runs',)
('Batsman_Scored',)
('Batting_Style',)
('Bowling_Style',)
('Country',)
('Season',)
('City',)
('Outcome',)
('Win_By',)
('Wicket_Taken',)
('Venue',)
('Extra_Type',)
('Out_Type',)
('Toss_Decision',)
('Umpire',)
('Team',)
('Ball_by_Ball',)
('sysdiagrams',)
('sqlite_sequence',)
('Match',)
('Rolee',)
('Player_Match',)


In [4]:
player_df = pd.read_sql_query("select * from player", conn)
player_df["DOB"] = pd.to_datetime(player_df["DOB"])
player_df.set_index("Player_Id", inplace=True)

player_df.head()

,Player_Name,DOB,Batting_hand,Bowling_skill,Country_Name
Player_Id,,,,,
1,SC Ganguly,1972-07-08,1,1.0,1
2,BB McCullum,1981-09-27,2,1.0,4
3,RT Ponting,1974-12-19,2,1.0,5
4,DJ Hussey,1977-07-15,2,2.0,5
5,Mohammad Hafeez,1980-10-17,2,2.0,6


In [5]:
player_df.isna().sum()

Player_Name       0
DOB               0
Batting_hand      0
Bowling_skill    43
Country_Name      0
dtype: int64

Now to clean the data.
First we notice that there are 43 players whose bowling skill entries are missing, so we fill them with 0s for now

In [6]:
player_df.fillna({"Bowling_skill":0}, inplace=True)
player_df.isnull().sum()

Player_Name      0
DOB              0
Batting_hand     0
Bowling_skill    0
Country_Name     0
dtype: int64

In [7]:
player_df.duplicated().sum()

0

After filling in the missing bowling skills we check for duplicates, which there are none of.

In [8]:
country_df = pd.read_sql_query("select * from country", conn).set_index("Country_Id")
print(country_df)
print()
batting_df = pd.read_sql_query("SELECT * FROM Batting_Style", conn).set_index("Batting_Id")
print(batting_df)
print()
bowling_df = pd.read_sql_query("SELECT * FROM Bowling_Style", conn).set_index("Bowling_Id")
print(bowling_df)

            Country_Name
Country_Id              
1                  India
2           South Africa
3                  U.A.E
4            New Zealand
5              Australia
6               Pakistan
7              Sri Lanka
8            West Indies
9              Zimbabwea
10               England
11            Bangladesh
12           Netherlands

              Batting_hand
Batting_Id                
1            Left-hand bat
2           Right-hand bat

                     Bowling_skill
Bowling_Id                        
1                 Right-arm medium
2               Right-arm offbreak
3            Right-arm fast-medium
4                  Legbreak googly
5            Right-arm medium-fast
6             Left-arm fast-medium
7           Slow left-arm orthodox
8           Slow left-arm chinaman
9             Left-arm medium-fast
10                        Legbreak
11                  Right-arm fast
12                Right-arm bowler
13                 Left-arm medium
14             

In [9]:
bowling_df.head(14)

,Bowling_skill
Bowling_Id,
1,Right-arm medium
2,Right-arm offbreak
3,Right-arm fast-medium
4,Legbreak googly
5,Right-arm medium-fast
6,Left-arm fast-medium
7,Slow left-arm orthodox
8,Slow left-arm chinaman
9,Left-arm medium-fast


Now we have to de-normalize this table. This will involve taking the referenced tables and combining them with the player table. To that end, we examine those tables. These are the country names, batting hands, and bowling skill types referred to by player. Next we convert the Bowling_skill series in player_df to int for consistency:

In [10]:
player_df["Bowling_skill"] = player_df["Bowling_skill"].astype(int)
player_df.head(20)

,Player_Name,DOB,Batting_hand,Bowling_skill,Country_Name
Player_Id,,,,,
1,SC Ganguly,1972-07-08,1,1,1
2,BB McCullum,1981-09-27,2,1,4
3,RT Ponting,1974-12-19,2,1,5
4,DJ Hussey,1977-07-15,2,2,5
5,Mohammad Hafeez,1980-10-17,2,2,6
6,R Dravid,1973-01-11,2,2,1
7,W Jaffer,1978-02-16,2,2,1
8,V Kohli,1988-11-05,2,1,1
9,JH Kallis,1975-10-16,2,3,2


Now to combine the tables by replacing their reference ids with their corresponding values:

In [11]:
player_df["Country_Name"] = player_df["Country_Name"].replace(1, "India")\
    .replace(2, "South Africa").replace(3, "U.A.E").replace(4, "New Zealand")\
    .replace(5,"Australia").replace(6, "Pakistan").replace(7, "Sri Lanka")\
    .replace(8, "West Indies").replace(9, "Zimbabwea").replace(10, "England")\
    .replace(11, "Bangladesh").replace(12, "Netherlands")

In [12]:
player_df["Batting_hand"] = player_df["Batting_hand"].replace(1, "Left-hand bat")\
    .replace(2, "Right-hand bat")

In [13]:
player_df["Bowling_skill"] = player_df["Bowling_skill"].replace(1, "Right-arm medium")\
    .replace(2, "Right-arm offbreak").replace(3, "Right-arm fast-medium").replace(4, "Legbreak googly")\
    .replace(5,"Right-arm medium-fast").replace(6, "Left-arm fast-medium").replace(7, "Slow left-arm orthodox")\
    .replace(8, "Slow left-arm chinaman").replace(9, "Left-arm medium-fast").replace(10, "Legbreak")\
    .replace(11, "Right-arm fast").replace(12, "Right-arm bowler").replace(13, "Left-arm medium")\
    .replace(14, "Left-arm fast").replace(0, "Unspecified")

In [14]:
player_df.head(25)

,Player_Name,DOB,Batting_hand,Bowling_skill,Country_Name
Player_Id,,,,,
1,SC Ganguly,1972-07-08,Left-hand bat,Right-arm medium,India
2,BB McCullum,1981-09-27,Right-hand bat,Right-arm medium,New Zealand
3,RT Ponting,1974-12-19,Right-hand bat,Right-arm medium,Australia
4,DJ Hussey,1977-07-15,Right-hand bat,Right-arm offbreak,Australia
5,Mohammad Hafeez,1980-10-17,Right-hand bat,Right-arm offbreak,Pakistan
6,R Dravid,1973-01-11,Right-hand bat,Right-arm offbreak,India
7,W Jaffer,1978-02-16,Right-hand bat,Right-arm offbreak,India
8,V Kohli,1988-11-05,Right-hand bat,Right-arm medium,India
9,JH Kallis,1975-10-16,Right-hand bat,Right-arm fast-medium,South Africa
